In [ ]:
#hide
#default_exp config.manager_factory
from nbdev.showdoc import show_doc
import warnings
import os

warnings.filterwarnings('ignore')

if not os.path.exists('settings.ini'):
    os.chdir('../..')

# Manager factory

> Register a subclassed ExperimentManager to be used by other modules. 

import inspect

from hpsearch.examples.example_experiment_manager import ExampleExperimentManager

em = ExampleExperimentManager()

source_path = inspect.getfile(em.__class__)
inspect.getmodulename('ExampleExperimentManager')

In [ ]:
#export
import inspect
import shutil
import hpsearch
import os
import logging
import pickle
import pdb

experiment_manager = None

class ManagerFactory (object):
    def __init__ (self, allow_base_class=True, verbose=0):
        self.allow_base_class = allow_base_class
        self.logger = logging.getLogger("experiment_manager")
        if verbose > 1:
            self.logger.setLevel('DEBUG')
        self.obtain_paths()
        self.method = 1
        #print (f'using import method {self.method}')
        #pdb.set_trace()
        
    def register_manager (self, experiment_manager_to_register):
        global experiment_manager
        experiment_manager = experiment_manager_to_register
        
    def obtain_paths (self):
        destination_path_folder = os.path.dirname (hpsearch.__file__)
        destination_path_folder = f'{destination_path_folder}/app_config'
        destination_path_module = f'{destination_path_folder}/subclassed_manager.py'
        self.destination_path_folder = destination_path_folder
        self.destination_path_module = destination_path_module
        self.destination_path_import = f'{destination_path_folder}/subclassed_manager_import.py'
        self.class_two_module_file = f'{self.destination_path_folder}/class_two_module.pk'
        self.class_two_import_file = f'{self.destination_path_folder}/class_two_import.pk'
        self.class_two_base_file = f'{self.destination_path_folder}/class_two_base.pk'
        self.current_path = os.path.abspath(os.path.curdir)
        #print (f'current path: {self.current_path}')
        
    def determine_import_string (self, source_path, base_path, experiment_manager):
        if self.method==1:
                import_module_string = experiment_manager.__class__.__module__
        elif self.method==2:
            if source_path.startswith(base_path + '/'):
                split_list = source_path.split(base_path + '/')
                import_module_string = split_list[1]
                import_module_string = import_module_string.replace('/','.')
                import_module_string = import_module_string.replace('.py', '')
            else:
                self.logger.warning (f'current path {base_path} not found in source path {source_path}')
                import_module_string = 'hpsearch.app_config.subclassed_manager'
        self.import_module_string = import_module_string
        return import_module_string
        
    def write_manager (self, experiment_manager):
        name_subclass = experiment_manager.__class__.__name__
        try:
            source_path = inspect.getfile(experiment_manager.__class__)
            self.obtain_paths()
            import_module_string = self.determine_import_string (source_path, self.current_path, experiment_manager)
            self.write_manager_subclass (name_subclass, source_path, self.current_path, import_module_string)
            
            self.load_class_two_module ()
            self.class_two_module.update({name_subclass: source_path})
            self.class_two_import.update({name_subclass: import_module_string})
            self.class_two_base.update({name_subclass: self.current_path})
            pickle.dump (self.class_two_module, open(self.class_two_module_file, 'wb'))
            pickle.dump (self.class_two_import, open(self.class_two_import_file, 'wb'))
            pickle.dump (self.class_two_base, open(self.class_two_base_file, 'wb'))
        except Exception as e:
            self.logger.warning (f'write_manager failed with exception {e}')
      
    def write_manager_subclass (self, name_subclass, source_path, base_path=None, import_module_string=None):
        if base_path is None:
            base_path = self.current_path
        os.makedirs(self.destination_path_folder, exist_ok=True)
        shutil.copy (source_path, self.destination_path_module)
        
        f = open (self.destination_path_import, 'wt')
        #if name_subclass not in self.class_two_module.keys():
        #f = open (self.destination_path_import, 'at')
        f.write ('import sys\n')
        f.write (f'sys.path.append("{base_path}")\n')
        #f.write (f'from hpsearch.app_config.subclassed_manager import {name_subclass} as Manager')
        f.write (f'from {import_module_string} import {name_subclass} as Manager')

        f.close()
    
    def load_class_two_module (self):
        if os.path.exists (self.class_two_module_file):
            self.class_two_module = pickle.load (open(self.class_two_module_file,'rb'))
        else:
            self.class_two_module = {}
            
        if os.path.exists (self.class_two_import_file):
            self.class_two_import = pickle.load (open(self.class_two_import_file,'rb'))
        else:
            self.class_two_import = {}
            self.logger.warning ('class2import not found, switching to original method')
            self.method = 2
            
        if os.path.exists (self.class_two_base_file):
            self.class_two_base = pickle.load (open(self.class_two_base_file,'rb'))
        else:
            self.class_two_base = {}
    
    def change_manager (self, name_subclass):
        self.previous_manager = self.get_experiment_manager ()
        self.obtain_paths()
        self.load_class_two_module ()
        if name_subclass not in self.class_two_module:
            raise ValueError (f'{name_subclass} not in dictionary class_two_module={self.class_two_module}')
        if name_subclass in self.class_two_base:
            base_path = self.class_two_base[name_subclass]
        else:
            base_path = self.current_path
        self.write_manager_subclass (name_subclass, self.class_two_module[name_subclass], base_path,
                                    self.class_two_import[name_subclass])
        self.reset_manager()
        self.import_written_manager()
        
    def switch_back (self):
        self.register_manager (self.previous_manager)
        self.write_manager (self.previous_manager)
        
    def print_current_manager (self):
        em = self.get_experiment_manager ()
        print (f'experiment manager registered: {em.__class__.__name__}')
        
    def list_subclasses (self):
        self.load_class_two_module ()
        print (f'subclasses: {self.class_two_module.keys()}')
        self.print_current_manager()
    
    def import_written_manager (self):
        global experiment_manager
        try:
            import hpsearch.app_config.subclassed_manager_import as subclass_module
            from importlib import reload
            reload (subclass_module)
            
            from hpsearch.app_config.subclassed_manager_import import Manager
            em = Manager()
            self.logger.debug ('returning subclassed manager')
        except ImportError:
            if not self.allow_base_class:
                raise ImportError (f'it was not possible to import subclassed manager, and allow_base_class=False')
            self.logger.debug ('importing base class ExperimentManager')
            from hpsearch.experiment_manager import ExperimentManager
            em = ExperimentManager()
        experiment_manager = em
        
    
    def get_experiment_manager (self):
        
        if experiment_manager is not None:
            em = experiment_manager
            self.logger.debug ('returning registered experiment manager')
        else:
            self.logger.debug ('experiment manager not registered yet, importing experiment manager')
            self.import_written_manager()
            em = self.get_experiment_manager ()
            
        self.logger.debug (f'returning experiment manager {em}')
        return em
    
    def reset_manager (self):
        self.register_manager (None)
        
    def set_base_manager (self):
        from hpsearch.experiment_manager import ExperimentManager
        em = ExperimentManager()
        self.register_manager (em)
    
    def delete_and_reset_all (self):
        self.obtain_paths()
        self.logger.debug (self.destination_path_module)
        if os.path.exists(self.destination_path_module):
            self.logger.debug ('deleting')
            os.remove(self.destination_path_module)
        if os.path.exists(self.destination_path_import):
            self.logger.debug ('deleting')
            os.remove(self.destination_path_import)
            
        self.load_class_two_module ()
        if os.path.exists(self.class_two_module_file):
            self.logger.debug (f'deleting {self.class_two_module_file}')
            os.remove(self.class_two_module_file)
            
        if os.path.exists(self.class_two_import_file):
            self.logger.debug (f'deleting {self.class_two_import_file}')
            os.remove(self.class_two_import_file)
            
        if os.path.exists(self.class_two_base_file):
            self.logger.debug (f'deleting {self.class_two_base_file}')
            os.remove(self.class_two_base_file)
            
        self.set_base_manager ()

The method `get_experiment_manager` returns the base class ExperimentManager initially, unless a subclass definition has been stored by calling the `write_manager` method

In [ ]:
factory = ManagerFactory ()
factory.delete_and_reset_all()
em = factory.get_experiment_manager()
assert em.__class__.__name__ == 'ExperimentManager'

current path: /mnt/athena/hpsearch
current path: /mnt/athena/hpsearch
current path: /mnt/athena/hpsearch


We can write the definition of a desired subclass of ExperimentManager with the method `write_manager`. After writing this definition, the object manager_factory will return an instance of the same subclass for any module that imports ManagerFactory. Let us check this with one example. We will use the subclass `ExampleExperimentManager` as an example, and check that the module file exists after calling the method `write_manager`.

In [ ]:
from hpsearch.examples.example_experiment_manager import ExampleExperimentManager

# we check that the file does not exist before writting it
assert not os.path.exists(factory.destination_path_module)

# we write new module and check that the file exists afterwards
em = ExampleExperimentManager()
factory.register_manager (em)
factory.write_manager (em)
assert os.path.exists(factory.destination_path_module)

# we check that the written module corresponds to the class ExampleExperimentManager
from hpsearch.app_config.subclassed_manager_import import Manager
em2 = Manager()
assert Manager.__name__ == 'ExampleExperimentManager'
assert em2.__class__.__name__ == em.__class__.__name__
assert em.get_default_parameters({}) == em2.get_default_parameters({})

current path: /mnt/athena/hpsearch
hpsearch.examples.example_experiment_manager
current path: /mnt/athena/hpsearch
current path: /mnt/athena/hpsearch


We do the same thing through the base class `ExperimentManager`

In [ ]:
em.register_and_store_subclassed_manager()

registering
hpsearch.examples.example_experiment_manager
current path: /mnt/athena/hpsearch


After writting the subclass, the method `get_experiment_manager` returns an instance of this subclass. This allows command-line scripts to use functions of the subclass indicated by other modules. 

In [ ]:
em2 = factory.get_experiment_manager()
assert em2.__class__.__name__ == em.__class__.__name__

The method `get_experiment_manager` returns a registered experiment_manager

In [ ]:
from hpsearch.experiment_manager import ExperimentManager
em = ExperimentManager()
factory.register_manager(em)
em2 = factory.get_experiment_manager()
assert em2.__class__.__name__ == 'ExperimentManager'

em = ExampleExperimentManager()
factory.register_manager(em)
em2 = factory.get_experiment_manager()
assert em2.__class__.__name__ == 'ExampleExperimentManager'

factory.register_manager(None)
em2 = factory.get_experiment_manager()
assert em2.__class__.__name__ == 'ExampleExperimentManager'

current path: /mnt/athena/hpsearch
current path: /mnt/athena/hpsearch
current path: /mnt/athena/hpsearch


In [ ]:
#hide
from nbdev.export import *
notebook2script(recursive=True)

Converted hpconfig.ipynb.
Converted manager_factory.ipynb.
Converted complex_dummy_experiment_manager.ipynb.
Converted dummy_experiment_manager.ipynb.
Converted example_experiment.ipynb.
Converted example_experiment_manager.ipynb.
Converted experiment_manager.ipynb.
Converted index.ipynb.
Converted change_manager.ipynb.
Converted metric_visualization.ipynb.
Converted print_parameters.ipynb.
Converted query.ipynb.
Converted remove_experiments.ipynb.
Converted experiment_utils.ipynb.
Converted organize_experiments.ipynb.
Converted resume_from_checkpoint.ipynb.
Converted experiment_visualization.ipynb.
Converted plot_utils.ipynb.


We check that a module registered in one module is used in another one. 

In [ ]:
del ManagerFactory
del experiment_manager
from hpsearch.config.manager_factory import ManagerFactory

factory = ManagerFactory()
em = factory.get_experiment_manager()
assert em.__class__.__name__ == 'ExampleExperimentManager'

defaults=em.get_default_parameters({})

current path: /mnt/athena/hpsearch


We check that the defaults are removed properly by using remove_defaults

In [ ]:
from hpsearch.utils.experiment_utils import remove_defaults

removed=remove_defaults(defaults)
assert removed=={}

removed=remove_defaults({'my_first':50, 'my_second':10, 'my_third':30})
assert removed=={'my_first': 50, 'my_third': 30}
removed

current path: /mnt/athena/hpsearch


{'my_first': 50, 'my_third': 30}

We check that no defaults are removed if the base class ExperimentManager is registered

In [ ]:
em3 = ExperimentManager(allow_base_class=True)
factory.register_manager(em3)
parameters = {'my_first':50, 'my_second':10, 'my_third':30}
parameters_copy = parameters.copy()
removed=remove_defaults(parameters_copy)
assert removed==parameters
removed

current path: /mnt/athena/hpsearch


{'my_first': 50, 'my_second': 10, 'my_third': 30}